In [1]:
import time

start_time = time.time()
%run ./welding/predict.py
end_time = time.time()

print(f"Execution time: {end_time - start_time:.2f} seconds")

D:\LaserWeldMonitor\cv_postprocessing
D:\LaserWeldMonitor\cv_postprocessing\welding\examples\images\10a.jpg
[2025/07/14 15:49:14] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
v 1
l 392

WARNING  Model does not support 'augment=True', reverting to single-scale prediction.
0: 480x640 1 weld, 50.7ms
Speed: 4.1ms preprocess, 50.7ms inference, 113.1ms postprocess per image at shape (1, 3, 480, 640)

WARNING  Model does not support 'augment=True', reverting to single-scale prediction.
0: 480x640 1 weld, 6.3ms
Speed: 2.4ms preprocess, 6.3ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)
t
 1121.0
le
 0.002551020408163265
T
 2.8596938775510203
{507: 'B', 515: 'B', 504: 'B', 509: 'B', 5011: 'C', 502: 'B'}
D:\LaserWeldMonitor\cv_postprocessing\welding\examples\images\10b.jpg
[2025/07/14 15:49:17] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
v 1
l 392



In [4]:
import pandas as pd
import re

# Read the CSV file into a DataFrame
df = pd.read_csv('welding/examples/output/props.csv')

# Define a function to split the 'key' into numeric and alphabetical parts
def split_key(key):
    match = re.match(r"(\d+)([a-zA-Z]+)", key)
    if match:
        return (int(match.group(1)), match.group(2))
    return (float('inf'), '')  # ssshandle case where the key does not match the expected pattern

# Apply the function to create a sorting key
df['sort_key'] = df['key'].apply(split_key)

# Sort the DataFrame based on the sorting key
df = df.sort_values('sort_key')

# Drop the temporary 'sort_key' column
df = df.drop(columns=['sort_key'])

# Save the sorted DataFrame back to a CSV
df.to_csv('welding/examples/output/props.csv', index=False)
